# Lab 4.4.6: Building Interactive Demos with Gradio

**Module:** 4.4 - Containerization & Cloud Deployment  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Beginner-Intermediate)

---

## Learning Objectives

By the end of this lab, you will:
- [ ] Create chat interfaces with Gradio
- [ ] Implement streaming responses for LLMs
- [ ] Add file upload capabilities for RAG
- [ ] Customize themes and styling
- [ ] Deploy to Hugging Face Spaces

---

## Prerequisites

- Basic Python knowledge
- Ollama running locally (optional, mock mode available)
- Hugging Face account (for deployment)

---

## Real-World Context

**Why Gradio?**

- Used by HuggingFace for all demo Spaces
- Create a shareable demo in minutes
- Perfect for showing ML models to stakeholders
- Built-in support for chat, image, audio, and more

**Use cases:**
- LLM chat interfaces
- Model comparison tools
- Prototype testing
- Stakeholder demos

---

## ELI5: What is Gradio?

> **Imagine you made an amazing cake (ML model)...**
>
> But to share it, people need to come to your kitchen and know how to operate your oven.
>
> **Gradio is like a bakery display case.** People can see your cake, try samples, and order more - all without touching your kitchen equipment!
>
> **In ML terms:**
> - Your model = The cake
> - Gradio interface = The display case
> - Users = Customers who can taste without knowing the recipe

In [ ]:
# Install Gradio if needed
# !pip install gradio>=4.0.0

import gradio as gr
print(f"Gradio version: {gr.__version__}")

---

## Understanding Our Demo Utilities Module

This curriculum provides a `demo_utils` module with classes for building LLM interfaces. Let's understand what it offers before using it.

### LLM Client Classes

| Class | Description | Use Case |
|-------|-------------|----------|
| `StreamingLLMClient` | Client for real LLM backends | Production with Ollama/OpenAI |
| `MockLLMClient` | Simulated LLM for testing | Development/demos without GPU |

### MockLLMClient

A simulated LLM client for testing without a real model:

```python
mock_client = MockLLMClient(
    responses=["Response 1", "Response 2"],  # Cycle through these
    delay_per_token=0.02,  # Simulate streaming delay
)
```

**Methods:**
- `stream_chat(messages)`: Generator yielding response tokens
- `chat(messages)`: Return complete response

### StreamingLLMClient

A client for real LLM backends:

```python
client = StreamingLLMClient(
    model="qwen3:8b",
    backend="ollama",  # or "openai"
    base_url="http://localhost:11434",
)
```

### Message Class

Represents a chat message:

```python
from scripts.demo_utils import Message
msg = Message(role="user", content="Hello!")
# role can be: "user", "assistant", or "system"
```

### Helper Functions

| Function | Description |
|----------|-------------|
| `create_gradio_chat_interface(client, title, ...)` | Create a Gradio ChatInterface |
| `generate_gradio_space_config(title, sdk, ...)` | Generate HuggingFace Spaces README |

In [ ]:
# Import our demo utilities
import sys
sys.path.insert(0, '..')

from scripts.demo_utils import (
    StreamingLLMClient,
    MockLLMClient,
    Message,
    create_gradio_chat_interface,
    generate_gradio_space_config,
)

print("Demo utilities loaded!")

---

## Part 1: Your First Gradio Interface

Let's start with the simplest possible Gradio app.

In [ ]:
# Simple function to greet
def greet(name):
    return f"Hello, {name}! Welcome to Gradio!"

# Create interface
demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(label="Your Name"),
    outputs=gr.Textbox(label="Greeting"),
    title="Hello World",
    description="Enter your name to get a greeting!",
)

# Launch (in notebook, this creates an iframe)
demo.launch(share=False, height=400)

### What Just Happened?

1. `gr.Interface` wraps your function with a UI
2. `inputs` defines what the user provides
3. `outputs` defines what gets displayed
4. `launch()` starts a local web server

**Pro tip:** Use `share=True` to get a public URL (temporary, 72 hours)

---

## Part 2: Building a Chat Interface

Gradio has a built-in `ChatInterface` for conversational AI.

In [ ]:
# Create a mock LLM client for testing
mock_client = MockLLMClient(
    responses=[
        "Hello! I'm a simulated AI assistant running on DGX Spark. How can I help you today?",
        "That's a great question! Let me think about it...\n\nBased on my training, I would say that machine learning is the process of teaching computers to learn from data rather than being explicitly programmed.",
        "Here's a simple Python example:\n\n```python\ndef fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)\n```\n\nThis recursive function calculates Fibonacci numbers!",
    ],
    delay_per_token=0.02,  # Simulate streaming
)

print("Mock client created!")

In [ ]:
# Build a chat interface with streaming
def chat_with_llm(message, history):
    """
    Chat function for Gradio.
    
    Args:
        message: User's current message
        history: List of [user_msg, assistant_msg] pairs
    
    Yields:
        Partial response for streaming effect
    """
    # Convert history to Message objects
    messages = []
    for user_msg, assistant_msg in history:
        messages.append(Message("user", user_msg))
        messages.append(Message("assistant", assistant_msg))
    messages.append(Message("user", message))
    
    # Stream response
    full_response = ""
    for chunk in mock_client.stream_chat(messages):
        full_response += chunk
        yield full_response

# Create chat interface
chat_demo = gr.ChatInterface(
    fn=chat_with_llm,
    title="AI Chat Demo",
    description="Chat with a simulated LLM (replace with real model for production)",
    examples=[
        "Hello! What can you do?",
        "Explain machine learning in simple terms",
        "Write a Python function for Fibonacci numbers",
    ],
    theme=gr.themes.Soft(),
)

chat_demo.launch(share=False, height=600)

---

## Part 3: Connecting to a Real LLM

Let's connect to Ollama (if running) or use the mock client.

In [ ]:
# Try to connect to Ollama
import requests

def check_ollama():
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        if response.status_code == 200:
            models = response.json().get("models", [])
            return [m["name"] for m in models]
    except:
        pass
    return []

available_models = check_ollama()

if available_models:
    print(f"Ollama is running with models: {available_models}")
    # Use real client
    llm_client = StreamingLLMClient(
        model=available_models[0],
        backend="ollama",
    )
else:
    print("Ollama not available, using mock client")
    llm_client = mock_client

In [ ]:
# Create chat interface with real/mock client
def chat_function(message, history):
    messages = []
    for user_msg, assistant_msg in history:
        messages.append(Message("user", user_msg))
        messages.append(Message("assistant", assistant_msg))
    messages.append(Message("user", message))
    
    full_response = ""
    for chunk in llm_client.stream_chat(messages):
        full_response += chunk
        yield full_response

# Use the helper function
demo = create_gradio_chat_interface(
    client=llm_client,
    title="DGX Spark AI Assistant",
    description=f"Powered by {'Ollama: ' + available_models[0] if available_models else 'Mock LLM'}",
    examples=[
        "What is the DGX Spark?",
        "Explain GPU memory management",
        "Write a CUDA kernel example",
    ],
    theme="soft",
)

demo.launch(share=False, height=600)

---

## Part 4: Adding File Upload for RAG

Let's add document upload capability for RAG-style applications.

In [ ]:
# RAG-enabled chat interface

# Store uploaded documents (in memory for demo)
uploaded_docs = []

def process_document(file):
    """Process uploaded document."""
    global uploaded_docs
    
    if file is None:
        return "No file uploaded"
    
    # Read file content
    try:
        with open(file.name, 'r') as f:
            content = f.read()
        
        uploaded_docs.append({
            "name": file.name.split("/")[-1],
            "content": content[:5000],  # Limit for demo
        })
        
        return f"Uploaded: {file.name.split('/')[-1]} ({len(content)} characters)"
    except Exception as e:
        return f"Error: {str(e)}"

def chat_with_context(message, history):
    """Chat with document context."""
    # Build context from uploaded docs
    context = ""
    if uploaded_docs:
        context = "\n\nUploaded documents:\n"
        for doc in uploaded_docs[-3:]:  # Last 3 docs
            context += f"\n--- {doc['name']} ---\n{doc['content'][:1000]}...\n"
    
    # Build messages with context
    messages = []
    if context:
        messages.append(Message("system", f"Use this context to answer questions:{context}"))
    
    for user_msg, assistant_msg in history:
        messages.append(Message("user", user_msg))
        messages.append(Message("assistant", assistant_msg))
    messages.append(Message("user", message))
    
    # Stream response
    full_response = ""
    for chunk in llm_client.stream_chat(messages):
        full_response += chunk
        yield full_response

# Create interface with file upload
with gr.Blocks(theme=gr.themes.Soft()) as rag_demo:
    gr.Markdown("# RAG Chat Demo")
    gr.Markdown("Upload documents and ask questions about them.")
    
    with gr.Row():
        with gr.Column(scale=1):
            file_upload = gr.File(
                label="Upload Document",
                file_types=[".txt", ".md", ".py"],
            )
            upload_status = gr.Textbox(label="Status", interactive=False)
            file_upload.change(process_document, file_upload, upload_status)
            
        with gr.Column(scale=3):
            chatbot = gr.ChatInterface(
                fn=chat_with_context,
                examples=[
                    "What's in the uploaded document?",
                    "Summarize the main points",
                    "What are the key concepts?",
                ],
            )

rag_demo.launch(share=False, height=700)

---

## Part 5: Custom Themes and Styling

In [ ]:
# Available built-in themes
themes = [
    "default",
    "soft",
    "glass",
    "monochrome",
]

print("Available Gradio themes:")
for theme in themes:
    print(f"  - gr.themes.{theme.capitalize()}()")

In [ ]:
# Custom theme example
custom_theme = gr.themes.Soft(
    primary_hue="green",
    secondary_hue="blue",
    neutral_hue="slate",
).set(
    button_primary_background_fill="#76b900",  # NVIDIA green
    button_primary_background_fill_hover="#5a8f00",
)

# Custom CSS
custom_css = """
.gradio-container {
    max-width: 900px !important;
}
.message {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
"""

# Create themed interface
themed_demo = gr.ChatInterface(
    fn=chat_function,
    title="NVIDIA DGX Spark Assistant",
    description="Powered by DGX Spark with 128GB unified memory",
    theme=custom_theme,
    css=custom_css,
    examples=[
        "What makes DGX Spark special?",
        "How much GPU memory is available?",
    ],
)

themed_demo.launch(share=False, height=500)

---

## Part 6: Deploying to Hugging Face Spaces

Hugging Face Spaces provides free hosting for Gradio apps!

In [ ]:
# Generate Spaces configuration
config = generate_gradio_space_config(
    title="DGX Spark AI Demo",
    sdk="gradio",
    sdk_version="4.0",
    requirements=["transformers", "torch", "accelerate"],
)

print("README.md for Hugging Face Spaces:")
print("=" * 60)
print(config)

In [ ]:
# Create a complete Space-ready app
import os

os.makedirs("../docker-examples/gradio-space", exist_ok=True)

# Save README.md
with open("../docker-examples/gradio-space/README.md", "w") as f:
    f.write(config)

# Save requirements.txt
requirements = """gradio>=4.0.0
transformers>=4.37.0
torch>=2.0.0
accelerate>=0.25.0
requests>=2.28.0
"""

with open("../docker-examples/gradio-space/requirements.txt", "w") as f:
    f.write(requirements)

# Save app.py
app_code = '''"""Gradio App for Hugging Face Spaces."""

import gradio as gr
from transformers import pipeline
import os

# Load model (small model for demo)
generator = pipeline("text-generation", model="gpt2")

def chat(message, history):
    """Chat function."""
    # Build prompt from history
    prompt = ""
    for user_msg, assistant_msg in history:
        prompt += f"User: {user_msg}\\nAssistant: {assistant_msg}\\n"
    prompt += f"User: {message}\\nAssistant:"
    
    # Generate response
    response = generator(
        prompt,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
    )[0]["generated_text"]
    
    # Extract just the assistant's response
    assistant_response = response.split("Assistant:")[-1].strip()
    
    return assistant_response

# Create interface
demo = gr.ChatInterface(
    fn=chat,
    title="DGX Spark AI Demo",
    description="A demo chatbot deployed from DGX Spark",
    examples=[
        "Hello! How are you?",
        "Tell me a joke",
        "What is machine learning?",
    ],
    theme=gr.themes.Soft(),
)

if __name__ == "__main__":
    demo.launch()
'''

with open("../docker-examples/gradio-space/app.py", "w") as f:
    f.write(app_code)

print("Created Hugging Face Space files:")
print("  - docker-examples/gradio-space/README.md")
print("  - docker-examples/gradio-space/requirements.txt")
print("  - docker-examples/gradio-space/app.py")
print("\nTo deploy:")
print("  1. Create a new Space on huggingface.co/spaces")
print("  2. Upload these files")
print("  3. Your app will be live in minutes!")

---

## Part 7: Advanced Features

In [ ]:
# Advanced Gradio app with multiple tabs
with gr.Blocks(theme=gr.themes.Soft()) as advanced_demo:
    gr.Markdown("# AI Playground")
    
    with gr.Tabs():
        # Tab 1: Chat
        with gr.Tab("Chat"):
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(label="Message", placeholder="Type a message...")
            clear = gr.Button("Clear")
            
            def respond(message, history):
                # Simple echo for demo
                response = f"You said: {message}"
                history.append((message, response))
                return "", history
            
            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)
        
        # Tab 2: Settings
        with gr.Tab("Settings"):
            temperature = gr.Slider(
                minimum=0, maximum=2, value=0.7, step=0.1,
                label="Temperature"
            )
            max_tokens = gr.Slider(
                minimum=50, maximum=2048, value=256, step=50,
                label="Max Tokens"
            )
            system_prompt = gr.Textbox(
                label="System Prompt",
                value="You are a helpful AI assistant.",
                lines=3,
            )
        
        # Tab 3: Info
        with gr.Tab("About"):
            gr.Markdown("""
            ## About This Demo
            
            This is a Gradio demo showcasing:
            - Tabbed interfaces
            - Chat functionality
            - Settings management
            
            Built with Gradio and deployed from DGX Spark.
            """)

advanced_demo.launch(share=False, height=600)

---

## Common Mistakes

### Mistake 1: Blocking in Streaming Functions

```python
# BAD - No streaming, user waits for full response
def chat(message, history):
    response = model.generate(message)  # Long wait
    return response

# GOOD - Yield partial responses
def chat(message, history):
    partial = ""
    for token in model.stream_generate(message):
        partial += token
        yield partial  # User sees tokens appear
```

---

### Mistake 2: Not Handling Errors

```python
# BAD - Crashes on error
def chat(message, history):
    return model.generate(message)

# GOOD - Graceful error handling
def chat(message, history):
    try:
        return model.generate(message)
    except Exception as e:
        return f"Error: {str(e)}. Please try again."
```

---

## Checkpoint

You've learned:
- Creating simple and chat interfaces with Gradio
- Implementing streaming responses
- Adding file upload for RAG applications
- Customizing themes
- Deploying to Hugging Face Spaces

---

## Challenge (Optional)

Build a complete demo that:
1. Allows model selection from a dropdown
2. Shows token count and generation speed
3. Supports image input for multimodal models
4. Has a "compare models" mode

---

## Further Reading

- [Gradio Documentation](https://gradio.app/docs/)
- [Hugging Face Spaces](https://huggingface.co/docs/hub/spaces)
- [Gradio Components](https://gradio.app/docs/components)

---

## Cleanup

In [ ]:
# Close any running Gradio servers
gr.close_all()
print("All Gradio servers closed.")